# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [88]:
cities_df=pd.read_csv("../WeatherPy/output/cities.csv")
cities_df.head()

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Bluff,-46.6000,168.3333,34.81,79,100,2.24,NZ,1628083735
1,Nazarovo,56.0064,90.3914,62.73,94,90,1.68,RU,1628083736
2,Georgetown,5.4112,100.3354,84.88,82,20,2.30,MY,1628083736
3,Lalmohan,22.3384,90.7419,83.23,83,16,13.06,BD,1628083736
4,Barrow,71.2906,-156.7887,42.82,78,90,8.05,US,1628083737


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [28]:
# Store 'Lat' and 'Lng' into  locations 
locations= cities_df[["Lat","Long"]].astype(float)

#Convert humidity to float and store but handle any potential NaN value
cities_df=cities_df.dropna()
humidity=cities_df["Humidity"].astype(float)

In [35]:
#Create a heamap layer for humidity

#Set center point and zoom level first
center_coords=(32.74, -16.93)
fig=gmaps.figure(center=center_coords, zoom_level=1.5)
heat_layer=gmaps.heatmap_layer(locations, weights=humidity,
                              dissipating=False, max_intensity=100,
                              point_radius=2)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [86]:
ideal_weather_df=cities_df[(cities_df["Max Temp"]>70) &
                         (cities_df["Max Temp"]<90) &
                         (cities_df["Wind Speed"]<10) &
                         (cities_df["Humidity"]<40) &
                         (cities_df["Cloudiness"]<10)]
ideal_weather_df

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
30,Saint george,37.1041,-113.5841,85.35,36,1,3.00,US,1628083745
233,Shakawe,-18.3667,21.8500,83.53,13,1,5.88,BW,1628083811
263,Yumen,40.2833,97.2000,81.05,10,0,8.66,CN,1628083821
276,Crestline,34.2420,-117.2856,76.55,32,1,0.00,US,1628083825
393,Jiuquan,39.7432,98.5174,77.45,13,1,8.48,CN,1628083869
418,Manoel urbano,-8.8389,-69.2597,82.51,33,4,4.34,BR,1628083876
440,Feijo,-8.1642,-70.3536,82.60,33,7,4.05,BR,1628083883
465,Samfya,-11.3649,29.5565,79.65,22,0,2.55,ZM,1628083891
541,Urumqi,43.8010,87.6005,86.95,18,0,6.71,CN,1628083913


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [89]:
hotel_df=ideal_weather_df
hotel_df['Hotel Name']=""

hotel_df

C:\Users\Samantha\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
30,Saint george,37.1041,-113.5841,85.35,36,1,3.00,US,1628083745,
233,Shakawe,-18.3667,21.8500,83.53,13,1,5.88,BW,1628083811,
263,Yumen,40.2833,97.2000,81.05,10,0,8.66,CN,1628083821,
276,Crestline,34.2420,-117.2856,76.55,32,1,0.00,US,1628083825,
393,Jiuquan,39.7432,98.5174,77.45,13,1,8.48,CN,1628083869,
418,Manoel urbano,-8.8389,-69.2597,82.51,33,4,4.34,BR,1628083876,
440,Feijo,-8.1642,-70.3536,82.60,33,7,4.05,BR,1628083883,
465,Samfya,-11.3649,29.5565,79.65,22,0,2.55,ZM,1628083891,
541,Urumqi,43.8010,87.6005,86.95,18,0,6.71,CN,1628083913,


In [98]:
# find the closest hotel of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

for index, row in hotel_df.iterrows():
    
    #get lat, long from df
    lat=row["Lat"]
    long=row["Long"]
    
    # add location to params dict for each city
    params['location']=f"{lat},{long}"
    
    city=row["City"]
    
    # assemble the url and make API request
    print(f"Results received for {city}")
    response=requests.get(base_url,params=params).json()
    
    #extract results
    results=response['results']
    
    try:
        hotel_df.loc[index,'Hotel Name'] = response["results"][0]['name']
        
    except IndexError:
        print(f"**There is no lodging in {city}**")
        


Results received for Saint george
Results received for Shakawe
Results received for Yumen
**There is no lodging in Yumen**
Results received for Crestline
Results received for Jiuquan
Results received for Manoel urbano
Results received for Feijo
Results received for Samfya
Results received for Urumqi


In [83]:
hotel_df

,index,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,30,Saint george,37.1041,-113.5841,85.35,36,1,3.00,US,1628083745,Best Western Plus Abbey Inn
1,233,Shakawe,-18.3667,21.8500,83.53,13,1,5.88,BW,1628083811,Ditlhapi Guest House
2,263,Yumen,40.2833,97.2000,81.05,10,0,8.66,CN,1628083821,
3,276,Crestline,34.2420,-117.2856,76.55,32,1,0.00,US,1628083825,Sleepy Hollow Cabins & Hotel
4,393,Jiuquan,39.7432,98.5174,77.45,13,1,8.48,CN,1628083869,Jiuquan Hotel
5,418,Manoel urbano,-8.8389,-69.2597,82.51,33,4,4.34,BR,1628083876,Baixada Beira Rio
6,440,Feijo,-8.1642,-70.3536,82.60,33,7,4.05,BR,1628083883,Hotel e Restaurante Aҫai
7,465,Samfya,-11.3649,29.5565,79.65,22,0,2.55,ZM,1628083891,Chita Lodge Samfya
8,541,Urumqi,43.8010,87.6005,86.95,18,0,6.71,CN,1628083913,Hua Ling Grand Hotel Urumqi


In [69]:
markers=gmaps.marker_layer(hotel_locations)

fig.add_layer(markers)

fig


Figure(layout=FigureLayout(height='420px'))

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
